In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import StructType, StructField, StringType

from pydataset import data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pyspark
import multiprocessing
from pyspark.sql.functions import *

spark = SparkSession.builder.getOrCreate()

These exercises should go in a notebook or script named wrangle. Add, commit, and push your changes.

This exercises uses the case.csv, dept.csv, and source.csv files from the san antonio 311 call dataset.

# 1. Read the case, department, and source data into their own spark dataframes.

In [35]:
source = spark.read.csv("source.csv", sep=",", header=True, inferSchema=True)
case = spark.read.csv('case.csv', sep=',', header=True, inferSchema=True)
dept = spark.read.csv('dept.csv', sep=',', header=True, inferSchema=True)

# 2. Let's see how writing to the local disk works in spark:
    Write the code necessary to store the source data in both csv and json format, store these as sources_csv and sources_json
    Inspect your folder structure. What do you notice?

In [3]:
source.write.csv('sources_csv', mode='overwrite')
source.write.json('sources_json', mode='overwrite')

# 3. Inspect the data in your dataframes. Are the data types appropriate? Write the code necessary to cast the values to the appropriate types.

In [6]:
source.printSchema(), case.printSchema(), dept.printSchema()

root
 |-- source_id: string (nullable = true)
 |-- source_username: string (nullable = true)

root
 |-- case_id: integer (nullable = true)
 |-- case_opened_date: string (nullable = true)
 |-- case_closed_date: string (nullable = true)
 |-- SLA_due_date: string (nullable = true)
 |-- case_late: string (nullable = true)
 |-- num_days_late: double (nullable = true)
 |-- case_closed: string (nullable = true)
 |-- dept_division: string (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- SLA_days: double (nullable = true)
 |-- case_status: string (nullable = true)
 |-- source_id: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: integer (nullable = true)

root
 |-- dept_division: string (nullable = true)
 |-- dept_name: string (nullable = true)
 |-- standardized_dept_name: string (nullable = true)
 |-- dept_subject_to_SLA: string (nullable = true)



(None, None, None)

In [12]:
source.show(), source.printSchema()

+---------+--------------------+
|source_id|     source_username|
+---------+--------------------+
|   100137|    Merlene Blodgett|
|   103582|         Carmen Cura|
|   106463|     Richard Sanchez|
|   119403|      Betty De Hoyos|
|   119555|      Socorro Quiara|
|   119868| Michelle San Miguel|
|   120752|      Eva T. Kleiber|
|   124405|           Lori Lara|
|   132408|       Leonard Silva|
|   135723|        Amy Cardenas|
|   136202|    Michelle Urrutia|
|   136979|      Leticia Garcia|
|   137943|    Pamela K. Baccus|
|   138605|        Marisa Ozuna|
|   138650|      Kimberly Green|
|   138650|Kimberly Green-Woods|
|   138793| Guadalupe Rodriguez|
|   138810|       Tawona Martin|
|   139342|     Jessica Mendoza|
|   139344|        Isis Mendoza|
+---------+--------------------+
only showing top 20 rows

root
 |-- source_id: string (nullable = true)
 |-- source_username: string (nullable = true)



(None, None)

In [13]:
dept.show(), dept.printSchema()

+--------------------+--------------------+----------------------+-------------------+
|       dept_division|           dept_name|standardized_dept_name|dept_subject_to_SLA|
+--------------------+--------------------+----------------------+-------------------+
|     311 Call Center|    Customer Service|      Customer Service|                YES|
|               Brush|Solid Waste Manag...|           Solid Waste|                YES|
|     Clean and Green|Parks and Recreation|    Parks & Recreation|                YES|
|Clean and Green N...|Parks and Recreation|    Parks & Recreation|                YES|
|    Code Enforcement|Code Enforcement ...|  DSD/Code Enforcement|                YES|
|Code Enforcement ...|Code Enforcement ...|  DSD/Code Enforcement|                YES|
|Code Enforcement ...|                null|  DSD/Code Enforcement|                YES|
|   Dangerous Premise|Code Enforcement ...|  DSD/Code Enforcement|                YES|
|Dangerous Premise...|Code Enforcement ...|

(None, None)

In [11]:
case.show(), case.printSchema()

+----------+----------------+----------------+------------+---------+-------------------+-----------+----------------+--------------------+------------------+-----------+---------+--------------------+----------------+
|   case_id|case_opened_date|case_closed_date|SLA_due_date|case_late|      num_days_late|case_closed|   dept_division|service_request_type|          SLA_days|case_status|source_id|     request_address|council_district|
+----------+----------------+----------------+------------+---------+-------------------+-----------+----------------+--------------------+------------------+-----------+---------+--------------------+----------------+
|1014127332|     1/1/18 0:42|    1/1/18 12:29|9/26/20 0:42|       NO| -998.5087616000001|        YES|Field Operations|        Stray Animal|             999.0|     Closed| svcCRMLS|2315  EL PASO ST,...|               5|
|1014127333|     1/1/18 0:46|     1/3/18 8:11| 1/5/18 8:30|       NO|-2.0126041669999997|        YES|     Storm Water|Remova

(None, None)

# 1. How old is the latest (in terms of days past SLA) currently open issue? How long has the oldest (in terms of days since opened) currently opened issue been open?

In [46]:
case.select(case.case_closed).toPandas().value_counts()

case_closed
YES            823594
NO              18110
dtype: int64

In [51]:
case.where((case.case_closed == 'NO') & (case.num_days_late > 0)).select(min(case.num_days_late)).show()

+------------------+
|min(num_days_late)|
+------------------+
|       0.060810185|
+------------------+



In [52]:
case.where(case.case_closed == 'NO').select(max(case.num_days_late)).show()

+------------------+
|max(num_days_late)|
+------------------+
|       348.6458333|
+------------------+



# 2. How many Stray Animal cases are there?

In [19]:
case.select('service_request_type').groupBy('service_request_type').count().where(case.service_request_type == 'Stray Animal').show(truncate=False)

+--------------------+-----+
|service_request_type|count|
+--------------------+-----+
|Stray Animal        |26760|
+--------------------+-----+



# 3. How many service requests that are assigned to the Field Operations department (dept_division) are not classified as "Officer Standby" request type (service_request_type)?

In [27]:
case.select('service_request_type').where(case.dept_division == 'Field Operations').where(case.service_request_type != 'Officer Standby').count()

113902

# 4. Convert the council_district column to a string column.

In [36]:
case = case.select('*',case.council_district.cast('string').alias('council_district_str')).drop('council_district')
case.show()

+----------+----------------+----------------+------------+---------+-------------------+-----------+----------------+--------------------+------------------+-----------+---------+--------------------+--------------------+
|   case_id|case_opened_date|case_closed_date|SLA_due_date|case_late|      num_days_late|case_closed|   dept_division|service_request_type|          SLA_days|case_status|source_id|     request_address|council_district_str|
+----------+----------------+----------------+------------+---------+-------------------+-----------+----------------+--------------------+------------------+-----------+---------+--------------------+--------------------+
|1014127332|     1/1/18 0:42|    1/1/18 12:29|9/26/20 0:42|       NO| -998.5087616000001|        YES|Field Operations|        Stray Animal|             999.0|     Closed| svcCRMLS|2315  EL PASO ST,...|                   5|
|1014127333|     1/1/18 0:46|     1/3/18 8:11| 1/5/18 8:30|       NO|-2.0126041669999997|        YES|     St

# 5. Extract the year from the case_closed_date column.

In [53]:
df = case.withColumnRenamed("SLA_due_date", "case_due_date")

In [54]:
print("--- Before handling dates")
df.select("case_opened_date", "case_closed_date", "case_due_date").show(5)

fmt = "M/d/yy H:mm"
df = (
    df.withColumn("case_opened_date", to_timestamp("case_opened_date", fmt))
    .withColumn("case_closed_date", to_timestamp("case_opened_date", fmt))
    .withColumn("case_due_date", to_timestamp("case_opened_date", fmt))
)

print("--- After")
df.select("case_opened_date", "case_closed_date", "case_due_date").show(5)

--- Before handling dates
+----------------+----------------+-------------+
|case_opened_date|case_closed_date|case_due_date|
+----------------+----------------+-------------+
|     1/1/18 0:42|    1/1/18 12:29| 9/26/20 0:42|
|     1/1/18 0:46|     1/3/18 8:11|  1/5/18 8:30|
|     1/1/18 0:48|     1/2/18 7:57|  1/5/18 8:30|
|     1/1/18 1:29|     1/2/18 8:13| 1/17/18 8:30|
|     1/1/18 1:34|    1/1/18 13:29|  1/1/18 4:34|
+----------------+----------------+-------------+
only showing top 5 rows

--- After
+-------------------+-------------------+-------------------+
|   case_opened_date|   case_closed_date|      case_due_date|
+-------------------+-------------------+-------------------+
|2018-01-01 00:42:00|2018-01-01 00:42:00|2018-01-01 00:42:00|
|2018-01-01 00:46:00|2018-01-01 00:46:00|2018-01-01 00:46:00|
|2018-01-01 00:48:00|2018-01-01 00:48:00|2018-01-01 00:48:00|
|2018-01-01 01:29:00|2018-01-01 01:29:00|2018-01-01 01:29:00|
|2018-01-01 01:34:00|2018-01-01 01:34:00|2018-01-01 01:

In [55]:
df.select(year(df.case_closed_date)).show()

+----------------------+
|year(case_closed_date)|
+----------------------+
|                  2018|
|                  2018|
|                  2018|
|                  2018|
|                  2018|
|                  2018|
|                  2018|
|                  2018|
|                  2018|
|                  2018|
|                  2018|
|                  2018|
|                  2018|
|                  2018|
|                  2018|
|                  2018|
|                  2018|
|                  2018|
|                  2018|
|                  2018|
+----------------------+
only showing top 20 rows



# 6. Convert num_days_late from days to hours in new columns num_hours_late.

In [56]:
case.withColumn('num_hours_late', case.num_days_late / 24).show()

+----------+----------------+----------------+------------+---------+-------------------+-----------+----------------+--------------------+------------------+-----------+---------+--------------------+--------------------+--------------------+
|   case_id|case_opened_date|case_closed_date|SLA_due_date|case_late|      num_days_late|case_closed|   dept_division|service_request_type|          SLA_days|case_status|source_id|     request_address|council_district_str|      num_hours_late|
+----------+----------------+----------------+------------+---------+-------------------+-----------+----------------+--------------------+------------------+-----------+---------+--------------------+--------------------+--------------------+
|1014127332|     1/1/18 0:42|    1/1/18 12:29|9/26/20 0:42|       NO| -998.5087616000001|        YES|Field Operations|        Stray Animal|             999.0|     Closed| svcCRMLS|2315  EL PASO ST,...|                   5|  -41.60453173333334|
|1014127333|     1/1/18 

# 7. Join the case data with the source and department data.

In [65]:
case_source = case.join(source, on='source_id', how='left')
case_source_dept = case_source.join(dept, on='dept_division', how='left')
case_source_dept.show()

+----------------+---------+----------+----------------+----------------+------------+---------+-------------------+-----------+--------------------+------------------+-----------+--------------------+--------------------+---------------+--------------------+----------------------+-------------------+
|   dept_division|source_id|   case_id|case_opened_date|case_closed_date|SLA_due_date|case_late|      num_days_late|case_closed|service_request_type|          SLA_days|case_status|     request_address|council_district_str|source_username|           dept_name|standardized_dept_name|dept_subject_to_SLA|
+----------------+---------+----------+----------------+----------------+------------+---------+-------------------+-----------+--------------------+------------------+-----------+--------------------+--------------------+---------------+--------------------+----------------------+-------------------+
|Field Operations| svcCRMLS|1014127332|     1/1/18 0:42|    1/1/18 12:29|9/26/20 0:42|     

# 8. Are there any cases that do not have a request source?

In [68]:
case_source_dept.select(case_source_dept.source_username).toPandas().isna().any()

source_username    False
dtype: bool

In [69]:
case_source_dept.select(case_source_dept.source_username).toPandas().value_counts()

source_username       
CRM_Listener              195416
svcCRMSS                  113585
Terri Barron               19840
Nadine Olmos               19832
Mary A Torres              19570
Isis Mendoza               19238
Guadalupe Rodriguez        18735
svcCRMLS                   18445
Delilah Sanchez            18103
George Ponce               17556
San Juanita Villarreal     17506
Jessica Mendoza            17492
Norma K. Sotoodeh          17234
Tawona Martin              16173
Maria Pena                 15817
Chris Castro               15747
Shirley Sanchez            15607
Jessica West               15088
Yvonne Casias              15080
Andrea Alvarez             14984
Danita Mitchell            14955
Jose Acosta                14044
Kimberly Green-Woods       13565
Kimberly Green             13565
Debra Lara                 13481
Sylvia Sanchez             13076
Ronald S. Sorrell          11111
Maria Woodard              11010
Socorro Quiara             10656
Lori Lara           

# 9. What are the top 10 service request types in terms of number of requests?

In [87]:
case.select('service_request_type').groupBy('service_request_type').count().sort(desc('count')).show(10,truncate=False)

+--------------------------------+-----+
|service_request_type            |count|
+--------------------------------+-----+
|No Pickup                       |86855|
|Overgrown Yard/Trash            |65895|
|Bandit Signs                    |32910|
|Damaged Cart                    |30338|
|Front Or Side Yard Parking      |28794|
|Stray Animal                    |26760|
|Aggressive Animal(Non-Critical) |24882|
|Cart Exchange Request           |22024|
|Junk Vehicle On Private Property|21473|
|Pot Hole Repair                 |20616|
+--------------------------------+-----+
only showing top 10 rows



# 10. What are the top 10 service request types in terms of average days late?

In [85]:
case.groupBy('service_request_type').agg(round(mean(case.num_days_late),2).alias('avg_days_late')).sort(desc('avg_days_late')).show(10,truncate=False)

+--------------------------------------+-------------+
|service_request_type                  |avg_days_late|
+--------------------------------------+-------------+
|Zoning: Junk Yards                    |175.96       |
|Labeling for Used Mattress            |162.43       |
|Record Keeping of Used Mattresses     |154.0        |
|Signage Requied for Sale of Used Mattr|151.64       |
|Storage of Used Mattress              |142.11       |
|Zoning: Recycle Yard                  |135.93       |
|Donation Container Enforcement        |131.76       |
|License Requied Used Mattress Sales   |128.8        |
|Traffic Signal Graffiti               |77.9         |
|Complaint                             |72.52        |
+--------------------------------------+-------------+
only showing top 10 rows



# 11. Does number of days late depend on department?

In [89]:
case_source_dept.groupBy('dept_name').agg(round(mean(case_source_dept.num_days_late),2).alias('avg_days_late')).sort(desc('avg_days_late')).show(truncate=False)

+-------------------------+-------------+
|dept_name                |avg_days_late|
+-------------------------+-------------+
|null                     |135.93       |
|Customer Service         |59.74        |
|Development Services     |13.43        |
|Solid Waste Management   |-2.2         |
|Metro Health             |-4.91        |
|Parks and Recreation     |-5.25        |
|Trans & Cap Improvements |-20.61       |
|Code Enforcement Services|-38.7        |
|Animal Care Services     |-226.52      |
|City Council             |null         |
+-------------------------+-------------+



# 12. How do number of days late depend on department and request type?

You might have noticed that the latest date in the dataset is fairly far off from the present day. To account for this, replace any occurances of the current time with the maximum date from the dataset.

In [102]:
case_source_dept.groupBy('dept_name','service_request_type').agg(round(mean(case_source_dept.num_days_late),2).alias('avg_days_late')).sort(asc('dept_name'),desc('service_request_type'),desc('avg_days_late')).show(truncate=False)

+-------------------------+--------------------------------------+-------------+
|dept_name                |service_request_type                  |avg_days_late|
+-------------------------+--------------------------------------+-------------+
|null                     |Zoning: Recycle Yard                  |135.93       |
|Animal Care Services     |Trapped/Confined Animal               |0.25         |
|Animal Care Services     |Stray Animal                          |-998.81      |
|Animal Care Services     |Spay/Neuter Request Response          |-6.98        |
|Animal Care Services     |Public Nuisance(Own Animal)           |-2.2         |
|Animal Care Services     |Officer Standby                       |-0.07        |
|Animal Care Services     |Injured Animal(Critical)              |-0.07        |
|Animal Care Services     |City Council Animal Request           |-1.55        |
|Animal Care Services     |Animal Permits Request                |22.2         |
|Animal Care Services     |A

In [108]:
case_source_dept.groupBy('service_request_type').pivot('dept_name').mean('num_days_late').sort(asc('service_request_type')).show(150,truncate=False)

+----------------------------------------------+----+--------------------+------------+-------------------------+------------------+--------------------+------------------+--------------------+----------------------+------------------------+
|service_request_type                          |null|Animal Care Services|City Council|Code Enforcement Services|Customer Service  |Development Services|Metro Health      |Parks and Recreation|Solid Waste Management|Trans & Cap Improvements|
+----------------------------------------------+----+--------------------+------------+-------------------------+------------------+--------------------+------------------+--------------------+----------------------+------------------------+
|"Vacant Lot-City (12"")"                      |null|null                |null        |-45.21877967939106       |null              |null                |null              |null                |null                  |null                    |
|"Vacant Lot-City (48"")"       